# Implementation of SegNet on Drone Images 

This notebook recreates the procedure of data preperation and data loading and also initializing, training and evaluating SegNet on this data.

As the code is organized in modules, large portions of the produced code is not contained directly herein, but can be consulted by opening the `.py` files in the `src/`-folder.

NOTE: Because the full network takes upwards of 30 GB of memory to train and evaluate, a simplified version is used here. The full version can be accessed by setting the `use_simple` variable to `False`. Runtime evaluation is also disabled by default to save memory. It is explained how to enable it further down.

In [1]:
!echo "Make sure that you are in /src before running this notebook"
!pwd  # Prints current path (Linux and MacOS only)

Make sure that you are in /src before running this notebook
/home/asger/Desktop/alpha_cellari_imageseg/src


## Data preperation

Downloads and prepares data. This will take a number of minutes to run and requires a little over a gigabyte of storage.

In [3]:
!python -m pip install wget
!python data_prepper.py

2019-12-29 09:56:40.126691	Preparation of data with images of shape (512, 512, 3)

2019-12-29 09:56:40.126819	Downloading images...
2019-12-29 09:56:40.126883	Done downloading images to paths
                          	local_data/raw.png
                          	local_data/target.png

2019-12-29 09:56:40.126925	Loading images...
2019-12-29 09:56:42.401438	Done loading images
                          	Shapes: (6815, 5364, 3)
                          	Split: (0.83, 0.17)

2019-12-29 09:56:42.401571	Padding images...
2019-12-29 09:56:43.272500	Done padding images
                          	Shapes: (7168, 5632, 3)

2019-12-29 09:56:43.272657	Saving subimages to location local_data/imgs/{type}-{index}.png
2019-12-29 09:56:59.591820	Done saving subimages

2019-12-29 09:56:59.592804	Standardizing aerial image...
2019-12-29 09:56:59.594032	Done standardizing image

2019-12-29 09:56:59.594258	Squeezing target images to single channel...
2019-12-29 10:01:15.062393	Done creating target values

## Preparing network training


In [3]:
from logger import Logger
from train import Trainer
from augment import Augmenter, AugmentationConfig, flip_lr, flip_tb
import torch

print("Configuring parameters...")

# If True, a simpler version of segnet is used
use_simple = True

# Configuration of network architechture
architecture = {
		"kernel_size":  3,
		"padding": 1, 
		"stride": 1,
		"pool_dims": (2, 2),
		"probs": 0.1,
		"reduce_complexity": use_simple,
}

learning_rate = 1.5e-4
batch_size = 3
epochs = 3000

# Configuration of data augmentation
# Reducing cropsize significantly reduces memory usage and training time
augmentations = AugmentationConfig(
    augments =  [flip_lr, flip_tb],  
    cropsize = (350, 350),
    augment_p = [0.5, 0.5]
)

torch.cuda.empty_cache()
print("Done configuring")

Configuring parameters...
Done configuring


## Training the Network

The following code trains the network.

Because we had access to significant compute recourses, we did not optimize the loss calculations for memory. It is therefore not possible to run evaluation on most machines, so it is disabled by default. It can be enabled by setting the `with_loss` argument to `True`. The loss curve is available in the report.

In [4]:
logger = Logger("logs/train_run.log", "Running full training loop")
trainer = Trainer("local_data/prep_out.json", logger)
net = trainer.model_trainer(architecture, learning_rate, augmentations, epochs, batch_size, val_every = 25, with_loss = False, save_every = 500)
net.save('model')

2020-01-03 13:18:51.410141	Running full training loop

2020-01-03 13:18:52.340897	Augmentations: {'cropsize': (350, 350), 'augments': [<function flip_lr at 0x7f19f858ddd0>, <function flip_tb at 0x7f1a6ed6cc20>], 'augment_p': [0.5, 0.5]}
2020-01-03 13:18:52.341221	Criterion and optimizer: CrossEntropyLoss()
                          	Adam (
                          	Parameter Group 0
                          	    amsgrad: False
                          	    betas: (0.9, 0.999)
                          	    eps: 1e-08
                          	    lr: 0.00015
                          	    weight_decay: 0
                          	)
2020-01-03 13:18:52.540913	Train size: 69
                          	Eval size: 15
                          	Test size: 24

2020-01-03 13:18:52.541743	Neural network information
                          		Net(
                          	  (encoder1): EncoderBlock(
                          	    (encoder): Sequential(
                          	      (


## Evaluating the Network

In [ ]:
tester = Tester("local_data/prep_out.json", logger)
tester.test_model(net, "local_data/test")


## Reconstructing the image

In the following, a complete forwarding is carried out using oversampling to prevent borders at the croppings.

In [ ]:
full_forward(net, None, True, "local_data/full-forward.png")
